In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time

In [9]:
option = webdriver.ChromeOptions()
# option.add_argument('--headless')
svc = Service('chromedriver.exe')

#search query
q = 'sepeda lipat'
q = q.replace(" ", "%20")
excel_name= q.replace("%20","")
page = 1
maxpage = 5
url_list = []
#define list of url
for i in range(maxpage+1):
    url = 'https://www.tokopedia.com/search?navsource=&'+'&page=' + str(page) + "&q="+ q + "&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st=product"
    page= page+1
    url_list.append(url)

print(url_list)
# #get data from urls    
results = []
for url in url_list:
    driver = webdriver.Chrome(service=svc, options=option)
    driver.set_window_size(1300,800)
    driver.get(url)

    #loop for scrolling
    ranges = 500
    for i in range (0,7):
        end = ranges * i
        script = "window.scrollTo(0,"+str(end)+")"
        driver.execute_script(script)
        time.sleep(1)

    time.sleep(3)
    content = driver.page_source
    driver.save_screenshot(excel_name+str(page)+'.png')
    page = page+1
    driver.quit()
    results.append(BeautifulSoup(content,'html.parser'))

print(results[0])

['https://www.tokopedia.com/search?navsource=&&page=1&q=sepeda%20lipat&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st=product', 'https://www.tokopedia.com/search?navsource=&&page=2&q=sepeda%20lipat&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st=product', 'https://www.tokopedia.com/search?navsource=&&page=3&q=sepeda%20lipat&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st=product', 'https://www.tokopedia.com/search?navsource=&&page=4&q=sepeda%20lipat&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st=product', 'https://www.tokopedia.com/search?navsource=&&page=5&q=sepeda%20lipat&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st=product', 'https://www.tokopedia.com/search?navsource=&&page=6&q=sepeda%20lipat&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st=product']
<html data-rh="lang" lang="id"><head>
<title>Jual sepeda lipat | Tokopedia</title>
<meta charset="utf-8"/>
<meta content="initial-scale=1, minimum-scale=

In [17]:
i=1
base_url ='https://shopee.co.id'
list_nama, list_pictures, list_price, list_link, list_sold, list_city = [],[],[],[],[],[]
for data in results:
    for item in data.find_all('div', class_="css-12sieg3"):
        # print('Proses data ke-'+str(i))
        name = item.find('div', class_="prd_link-product-name css-svipq6").get_text()
        pictures = item.find('img')['src']
        price = item.find('div', class_="prd_link-product-price css-1ksb19c").get_text()
        price = price[2:]
        link = base_url + item.find('a')['href']
        sold = item.find('span', class_="prd_label-integrity css-1duhs3e")
        if sold != None:
            sold = sold.get_text().split()[1]
        city = item.find('span', class_="prd_link-shop-loc css-1kdc32b flip").get_text()

        list_nama.append(name)
        list_pictures.append(pictures)
        list_price.append(price)
        list_link.append(link)
        list_sold.append(sold)
        list_city.append(city)

        # print(item)
        i+=1
        # print('-------')
    
    # save to excel
    df = pd.DataFrame({'Nama':list_nama,'Price':list_price,'Sold':list_sold,'City':city, 'Pictures':list_pictures, 'Link':list_link})
    writer = pd.ExcelWriter(excel_name+".xlsx")
    df.to_excel(writer,'Sheet1', index=False)
    writer.save()